In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt             # Visualization
import seaborn as sns                       # Visualization
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import skew, kurtosis
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Import necessary modules for DL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Set visualization defaults for seaborn
sns.set(color_codes=True)

# Mount Google Drive (if using Google Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    colab_env = True
except ImportError:
    colab_env = False

# Load dataset
csv_path = '/content/drive/My Drive/CAMEL_ML/Full_dataset_Updated_Shah.csv' if colab_env else 'Full_dataset_Updated_Shah.csv'
df = pd.read_csv(csv_path)

# Convert date column to datetime and set as index
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.set_index('date', inplace=True)
else:
    raise ValueError("The dataset does not contain a 'date' column.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#BiLSTM

Three train/test procedure. Such as-
1. All gauges (without gauge ID as input): Model is trained on a subset of gauge stations (e.g., 70% of all) together and tested on aother unseen subset of gauge stations. Better to understand the spatial generalization capability.
2. Each site separately: Model trained on each site of a subset of all gauge stations and tested on another subset each site serately.
3. Train/test within sites: Model tarined/tested within each site.    

##All gauges (without gauge id)

In [ ]:
# Get unique gauge_ids
unique_gauge_ids = df['gauge_id'].unique()

# Define the number of samples for each set
num_train = 470
num_dev = 68
num_test = 136

# Randomly sample unique gauge_ids for train, dev, and test sets
train_gauge_ids = np.random.choice(unique_gauge_ids, num_train, replace=False)
remaining_gauge_ids = np.setdiff1d(unique_gauge_ids, train_gauge_ids)
dev_gauge_ids = np.random.choice(remaining_gauge_ids, num_dev, replace=False)
test_gauge_ids = np.setdiff1d(remaining_gauge_ids, dev_gauge_ids)

# Create the subsets
train_df = df[df['gauge_id'].isin(train_gauge_ids)]
dev_df = df[df['gauge_id'].isin(dev_gauge_ids)]
test_df = df[df['gauge_id'].isin(test_gauge_ids)]

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

In [ ]:
# Define a function to train on multiple sites and test on others using BiLSTM
def train_on_multiple_sites_bilstm(train_df, test_df, train_gauge_ids_sample, test_gauge_ids_sample, timesteps=20, epochs=40, batch_size=32):
    # Filter the dataset for the selected training sites
    train_df_sample = train_df[train_df['gauge_id'].isin(train_gauge_ids_sample)]

    # Filter the dataset for the selected testing sites
    test_df_sample = test_df[test_df['gauge_id'].isin(test_gauge_ids_sample)]

    # Initialize the StandardScaler and scale the features
    scaler_X = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(train_df_sample[features])
    X_test_scaled = scaler_X.transform(test_df_sample[features])

    # Prepare the target variables
    y_train = train_df_sample[target].values.ravel()
    y_test = test_df_sample[target].values.ravel()

    # Check if there are enough samples for BiLSTM to work
    if len(X_train_scaled) < timesteps or len(X_test_scaled) < timesteps:
        raise ValueError(f"Not enough samples for BiLSTM with timesteps={timesteps}.")

    # Define the number of features
    features_count = X_train_scaled.shape[1]

    # Reshape the training data for BiLSTM
    n_samples_train = X_train_scaled.shape[0]
    n_samples_test = X_test_scaled.shape[0]

    X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, features_count))
    X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, features_count))

    for i in range(timesteps, n_samples_train + 1):
        X_train_reshaped[i - timesteps] = X_train_scaled[i - timesteps:i]

    for i in range(timesteps, n_samples_test + 1):
        X_test_reshaped[i - timesteps] = X_test_scaled[i - timesteps:i]

    # Reshape the target variables accordingly
    y_train_reshaped = y_train[timesteps-1:]
    y_test_reshaped = y_test[timesteps-1:]

    # Build the BiLSTM model
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu', input_shape=(timesteps, features_count))))
    model.add(Dropout(0.2))
    model.add(Dense(units=32, activation='linear'))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

    # Train the BiLSTM model with early stopping
    history = model.fit(X_train_reshaped, y_train_reshaped,
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1,
                        validation_split=0.1,
                        callbacks=[early_stopping])

    # Make predictions on the train set
    y_train_pred = model.predict(X_train_reshaped)
    # Compute R² score for the train set
    r2_train = r2_score(y_train_reshaped, y_train_pred)

    # Make predictions on the test set
    y_test_pred = model.predict(X_test_reshaped)
    # Compute R² score for the test set
    r2_test = r2_score(y_test_reshaped, y_test_pred)

    print(f'Training R² score: {r2_train:.4f}')
    print(f'Testing R² score: {r2_test:.4f}')

    return r2_train, r2_test

# Example usage:
# Manually select sample of gauge stations for training from train_df
train_gauge_ids_sample = train_df['gauge_id'].unique()[:5]  # Manually select 5 gauges for training

# Manually select sample of gauge stations for testing from test_df
test_gauge_ids_sample = test_df['gauge_id'].unique()[:5]  # Manually select 5 gauges for testing

# Run the training and testing process
r2_train, r2_test = train_on_multiple_sites_bilstm(train_df, test_df, train_gauge_ids_sample, test_gauge_ids_sample)

##Each site separately

In [ ]:
# Get unique gauge_ids
unique_gauge_ids = df['gauge_id'].unique()

# Define the number of samples for each set
num_train = 470
num_dev = 68
num_test = 136

# Randomly sample unique gauge_ids for train, dev, and test sets
train_gauge_ids = np.random.choice(unique_gauge_ids, num_train, replace=False)
remaining_gauge_ids = np.setdiff1d(unique_gauge_ids, train_gauge_ids)
dev_gauge_ids = np.random.choice(remaining_gauge_ids, num_dev, replace=False)
test_gauge_ids = np.setdiff1d(remaining_gauge_ids, dev_gauge_ids)

# Create the subsets
train_df = df[df['gauge_id'].isin(train_gauge_ids)]
dev_df = df[df['gauge_id'].isin(dev_gauge_ids)]
test_df = df[df['gauge_id'].isin(test_gauge_ids)]

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

In [ ]:
# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

# Initialize the model training function
def train_bilstm_on_sites(train_df, train_gauge_ids_sample, timesteps=20, epochs=40, batch_size=32):
    # Initialize dictionary to store R² scores for each training gauge
    train_r2_scores = {}

    # Filter the dataset for the selected training sites
    for gauge_id in train_gauge_ids_sample:
        train_df_sample = train_df[train_df['gauge_id'] == gauge_id]

        # Initialize the StandardScaler and scale the features
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_df_sample[features])

        # Prepare the target variables
        y_train = train_df_sample[target].values.ravel()

        # Define the number of features
        features_count = X_train_scaled.shape[1]

        # Reshape the training data for BiLSTM
        n_samples_train = X_train_scaled.shape[0]
        X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, features_count))

        for i in range(timesteps, n_samples_train + 1):
            X_train_reshaped[i - timesteps] = X_train_scaled[i - timesteps:i]

        # Reshape the target variable accordingly
        y_train_reshaped = y_train[timesteps-1:]

        # Build the BiLSTM model
        model = Sequential()
        model.add(Bidirectional(LSTM(units=64, activation='relu', input_shape=(timesteps, features_count))))
        model.add(Dropout(0.2))
        model.add(Dense(units=32, activation='linear'))
        model.add(Dense(units=1))

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

        # Train the BiLSTM model with early stopping
        history = model.fit(X_train_reshaped, y_train_reshaped,
                            epochs=epochs,
                            batch_size=batch_size,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[early_stopping])

        # Make predictions on the train set
        y_train_pred = model.predict(X_train_reshaped)
        # Compute R² score for the train set
        train_r2_scores[gauge_id] = r2_score(y_train_reshaped, y_train_pred)

        print(f'Site {gauge_id}:')
        print(f'  Train R² score: {train_r2_scores[gauge_id]:.4f}')
        print('-' * 50)

    # Return the trained model and the training R² scores
    return model, train_r2_scores

### Training Code Block (on train_df) ###
train_gauge_ids_sample = train_df['gauge_id'].unique()[:5]  # Manually specify a batch of training gauges
trained_model, train_r2_scores = train_bilstm_on_sites(train_df, train_gauge_ids_sample)  # Train the model

### Testing Code Block (on test_df) ###
def process_gauges_for_testing(test_gauge_ids, df, features, target, trained_model, timesteps=20):
    # Initialize dictionary to store R² scores for each gauge
    test_r2_scores = {}

    # Process each gauge in the selected test_gauge_ids
    for gauge_id in test_gauge_ids:
        # Filter data for the current gauge
        test_site_data = df[df['gauge_id'] == gauge_id]

        # Initialize the StandardScaler and scale the features
        scaler_X = StandardScaler()
        X_test_scaled = scaler_X.fit_transform(test_site_data[features])

        # Prepare the target variables
        y_test = test_site_data[target].values.ravel()

        # Define the number of features
        features_count = X_test_scaled.shape[1]

        # Reshape the testing data for BiLSTM
        n_samples_test = X_test_scaled.shape[0]
        X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, features_count))

        for i in range(timesteps, n_samples_test + 1):
            X_test_reshaped[i - timesteps] = X_test_scaled[i - timesteps:i]

        # Reshape the target variable accordingly
        y_test_reshaped = y_test[timesteps-1:]

        # Make predictions on the test set using the trained model
        y_test_pred = trained_model.predict(X_test_reshaped)

        # Compute R² score for the test set
        test_r2_scores[gauge_id] = r2_score(y_test_reshaped, y_test_pred)

        print(f'Site {gauge_id}:')
        print(f'  Test R² score: {test_r2_scores[gauge_id]:.4f}')
        print('-' * 50)

    return test_r2_scores

# Manually specify a batch of gauge_ids to test on (select first 5 gauges from test_df)
test_gauge_ids_sample = test_df['gauge_id'].unique()[:5]

# Test the model on the selected gauges
test_r2_scores = process_gauges_for_testing(test_gauge_ids_sample, test_df, features, target, trained_model=trained_model)

# Create DataFrames for both training and testing R² scores
train_r2_df = pd.DataFrame(list(train_r2_scores.items()), columns=['gauge_id', 'train_r2'])
test_r2_df = pd.DataFrame(list(test_r2_scores.items()), columns=['gauge_id', 'test_r2'])

# Print the resulting DataFrames
print("Training R² scores:")
print(train_r2_df)
print("\nTest R² scores:")
print(test_r2_df)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 3141478.2500 - val_loss: 725628.6875
Epoch 2/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 980674.8125 - val_loss: 659284.0625
Epoch 3/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 790107.6250 - val_loss: 1054832.8750
Epoch 4/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 760232.6875 - val_loss: 727046.1875
Epoch 5/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 660410.5625 - val_loss: 525310.9375
Epoch 6/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 622757.0625 - val_loss: 463155.8438
Epoch 7/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 627583.2500 - val_loss: 704067.3750
Epoch 8/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 529487.6875 - val_loss: 508038.1562
Epoch 9/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 502639.4062 - val_loss: 377383.8438
Epoch 10/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 515702.4375 - val_loss: 406704.2812
Epoch 11/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6m

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


357/357 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 11119432.0000 - val_loss: 6647192.0000
Epoch 2/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 4555845.0000 - val_loss: 3151879.2500
Epoch 3/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 4354895.5000 - val_loss: 2400009.0000
Epoch 4/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 3028869.5000 - val_loss: 1992335.5000
Epoch 5/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 2654542.5000 - val_loss: 2299757.7500
Epoch 6/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 2489608.5000 - val_loss: 2249568.0000
Epoch 7/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 2457131.5000 - val_loss: 2732704.0000
Epoch 8/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 2357934.5000 - val_loss: 2950979.2500
Epoch 9/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 2044269.5000 - val_loss: 3044344.2500
Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 4.
397/397 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
Sit

...

357/357 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 870288.0625 - val_loss: 362955.8438
Epoch 2/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 525697.4375 - val_loss: 327626.9688
Epoch 3/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 547893.3750 - val_loss: 334060.8750
Epoch 4/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 331030.1562 - val_loss: 368615.9375
Epoch 5/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 474833.3750 - val_loss: 305213.9375
Epoch 6/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 436199.9375 - val_loss: 328617.8125
Epoch 7/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 237831.5312 - val_loss: 302320.0312
Epoch 8/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 294152.7188 - val_loss: 288056.0312
Epoch 9/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 305134.5312 - val_loss: 403442.2188
Epoch 10/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 270981.5000 - val_loss: 287128.2500
Epoch 11/40
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


359/359 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 1749208.2500 - val_loss: 918948.5000
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1195298.7500 - val_loss: 391554.6250
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 899787.9375 - val_loss: 452268.7188
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 936363.1250 - val_loss: 330598.1875
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 490590.7812 - val_loss: 363598.0625
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 594047.8750 - val_loss: 479161.1875
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 433730.9062 - val_loss: 436676.5000
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 512168.6250 - val_loss: 261258.0781
Epoch 9/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 448627.7500 - val_loss: 767158.2500
Epoch 10/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 401408.2812 - val_loss: 232028.7188
Epoch 11/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


359/359 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 364081.2812 - val_loss: 145956.5156
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 184883.4219 - val_loss: 269790.2500
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 139229.1562 - val_loss: 88672.3672
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 135747.5625 - val_loss: 92642.9219
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 121103.0938 - val_loss: 80183.6328
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 102426.7031 - val_loss: 88748.8516
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 94573.8828 - val_loss: 92058.6094
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 98196.1328 - val_loss: 81307.0234
Epoch 9/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 103546.3906 - val_loss: 79888.0156
Epoch 10/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 82882.7500 - val_loss: 79604.3594
Epoch 11/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 

##Train/test within sites

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Function to calculate NSE and PBIAS
def calculate_nse(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def calculate_pbias(y_true, y_pred):
    return 100 * np.sum(y_true - y_pred) / np.sum(y_true)

# Function to process a batch of gauge stations and run the BiLSTM model
def process_gauges_individually(df, gauge_ids_batch):
    # Initialize a list to store metrics for each site
    metrics = []

    # Process each site in the current batch
    for gauge_id in gauge_ids_batch:
        # Filter data for the current site
        site_data = df[df['gauge_id'] == gauge_id]

        # Determine the split index for 80% training data
        split_index = int(len(site_data) * 0.8)

        # Split the data into training and testing sets
        train_site_data = site_data.iloc[:split_index]
        test_site_data = site_data.iloc[split_index:]

        # Initialize the StandardScaler and scale the features
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(train_site_data[features])
        X_test_scaled = scaler_X.transform(test_site_data[features])

        # Prepare the target variables
        y_train = train_site_data[target].values.ravel()
        y_test = test_site_data[target].values.ravel()

        # Define the number of timesteps and features
        timesteps = 20  # Number of previous data points to consider
        features_count = X_train_scaled.shape[1]  # Number of features after transformation

        # Reshape the training data for BiLSTM
        n_samples_train = X_train_scaled.shape[0]
        n_samples_test = X_test_scaled.shape[0]

        X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, features_count))
        X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, features_count))

        for i in range(timesteps, n_samples_train + 1):
            X_train_reshaped[i - timesteps] = X_train_scaled[i - timesteps:i]

        for i in range(timesteps, n_samples_test + 1):
            X_test_reshaped[i - timesteps] = X_test_scaled[i - timesteps:i]

        # Reshape the target variable accordingly
        y_train_reshaped = y_train[timesteps-1:]
        y_test_reshaped = y_test[timesteps-1:]

        # Build the Simplified BiLSTM Model
        model = Sequential()
        model.add(Bidirectional(LSTM(units=32, activation='relu', input_shape=(timesteps, features_count))))  # Single BiLSTM layer
        model.add(Dense(units=1))  # Single dense output layer for regression

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Define early stopping criteria
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

        # Train the BiLSTM model with early stopping
        history = model.fit(X_train_reshaped, y_train_reshaped,
                            epochs=40,
                            batch_size=32,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[early_stopping])

        # Make predictions
        y_train_pred = model.predict(X_train_reshaped).flatten()
        y_test_pred = model.predict(X_test_reshaped).flatten()

        # Compute metrics for the train and test sets
        train_r2 = r2_score(y_train_reshaped, y_train_pred)
        test_r2 = r2_score(y_test_reshaped, y_test_pred)
        train_mse = mean_squared_error(y_train_reshaped, y_train_pred)
        test_mse = mean_squared_error(y_test_reshaped, y_test_pred)
        train_rmse = np.sqrt(train_mse)
        test_rmse = np.sqrt(test_mse)
        train_mae = mean_absolute_error(y_train_reshaped, y_train_pred)
        test_mae = mean_absolute_error(y_test_reshaped, y_test_pred)
        train_nse = calculate_nse(y_train_reshaped, y_train_pred)
        test_nse = calculate_nse(y_test_reshaped, y_test_pred)
        train_pbias = calculate_pbias(y_train_reshaped, y_train_pred)
        test_pbias = calculate_pbias(y_test_reshaped, y_test_pred)

        # Store the metrics
        metrics.append({
            'gauge_id': gauge_id,
            'train_r2': train_r2, 'test_r2': test_r2,
            'train_rmse': train_rmse, 'test_rmse': test_rmse,
            'train_mse': train_mse, 'test_mse': test_mse,
            'train_mae': train_mae, 'test_mae': test_mae,
            'train_nse': train_nse, 'test_nse': test_nse,
            'train_pbias': train_pbias, 'test_pbias': test_pbias
        })

        print(f"Gauge ID: {gauge_id} processed successfully.")

    return pd.DataFrame(metrics)

# Function to process gauge stations in chunks
def process_gauges_in_chunks(df, start_idx=0, chunk_size=30, results_file='results.csv'):
    try:
        results_df = pd.read_csv(results_file)
    except FileNotFoundError:
        results_df = pd.DataFrame(columns=['gauge_id', 'train_r2', 'test_r2', 'train_rmse', 'test_rmse',
                                           'train_mse', 'test_mse', 'train_mae', 'test_mae', 'train_nse',
                                           'test_nse', 'train_pbias', 'test_pbias'])

    unique_gauge_ids = df['gauge_id'].unique()
    total_gauges = len(unique_gauge_ids)

    gauge_ids_batch = unique_gauge_ids[start_idx:start_idx + chunk_size]
    print(f"Processing gauge stations {start_idx + 1} to {min(start_idx + chunk_size, total_gauges)}...")

    df_subset = df[df['gauge_id'].isin(gauge_ids_batch)]
    metrics_df = process_gauges_individually(df_subset, gauge_ids_batch)

    results_df = pd.concat([results_df, metrics_df], ignore_index=True)
    results_df.to_csv(results_file, index=False)

    print("Results saved to", results_file)

# Example usage:
features = ['dayl', 'prcp', 'srad', 'tmax', 'tmin', 'vp']
target = ['streamflow']

process_gauges_in_chunks(df, start_idx=630, chunk_size=50)

Processing gauge stations 631 to 674...
Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


287/287 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 184.8649 - val_loss: 34.1892
Epoch 2/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 72.6808 - val_loss: 28.1439
Epoch 3/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 58.2256 - val_loss: 22.8328
Epoch 4/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 57.0159 - val_loss: 21.9975
Epoch 5/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 50.2055 - val_loss: 22.7778
Epoch 6/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 50.2714 - val_loss: 20.0366
Epoch 7/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 40.3364 - val_loss: 33.1595
Epoch 8/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 43.2461 - val_loss: 17.4783
Epoch 9/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 44.1128 - val_loss: 21.8620
Epoch 10/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 32.6931 - val_loss: 26.3166
Epoch 11/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 33.7978 - val_loss: 19.1602
Epoch 12/40
287/287 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 510215.8750 - val_loss: 83020.3281
Epoch 2/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 121404.5859 - val_loss: 52934.3555
Epoch 3/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 99147.5078 - val_loss: 59573.1914
Epoch 4/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 100232.5469 - val_loss: 68780.4375
Epoch 5/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 95612.9453 - val_loss: 44216.5586
Epoch 6/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 89830.1953 - val_loss: 39608.3125
Epoch 7/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 82090.4375 - val_loss: 55214.2578
Epoch 8/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 91623.9922 - val_loss: 53348.9453
Epoch 9/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 84264.1562 - val_loss: 42393.1406
Epoch 10/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 78378.9375 - val_loss: 52668.3633
Epoch 11/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 82877

...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


259/259 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 271.9365 - val_loss: 24.3898
Epoch 2/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 100.0170 - val_loss: 21.5847
Epoch 3/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 93.0433 - val_loss: 30.4241
Epoch 4/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 149.9683 - val_loss: 42.7102
Epoch 5/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 159.5242 - val_loss: 28.1284
Epoch 6/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 77.6297 - val_loss: 17.4848
Epoch 7/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 102.5667 - val_loss: 28.4020
Epoch 8/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 79.4059 - val_loss: 22.3239
Epoch 9/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 48.4806 - val_loss: 58.6928
Epoch 10/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 118.7457 - val_loss: 22.2180
Epoch 11/40
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 64.8912 - val_loss: 20.7088
Epoch 11: early stopping
Restoring

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


288/288 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 12240746.0000 - val_loss: 2558488.7500
Epoch 2/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2564337.5000 - val_loss: 5663285.0000
Epoch 3/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2619745.2500 - val_loss: 2026106.2500
Epoch 4/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2138580.7500 - val_loss: 2265297.0000
Epoch 5/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2063528.3750 - val_loss: 3074482.7500
Epoch 6/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1930724.5000 - val_loss: 3301125.5000
Epoch 7/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1870997.0000 - val_loss: 2917657.5000
Epoch 8/40
288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2156882.5000 - val_loss: 2173956.0000
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 3.
319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Gauge ID: 14400000 processed successfully.
Results saved to re

<ipython-input-11-784e18e5b3e7>:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, metrics_df], ignore_index=True)


##All gauges (with gauges ids as input)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

# Filter the dataset to include only the first 10 unique gauge_ids
first_10_gauge_ids = df['gauge_id'].unique()[:10]
df_filtered = df[df['gauge_id'].isin(first_10_gauge_ids)]

# Convert gauge_id to categorical for the filtered data
df_filtered['gauge_id'] = df_filtered['gauge_id'].astype('category')

# Split the filtered data into train and test sets based on gauge_id
train_gauge_ids = df_filtered['gauge_id'].unique()[:8]  # First 8 sites for training
test_gauge_ids = df_filtered['gauge_id'].unique()[8:]   # Remaining 2 sites for testing

train_df = df_filtered[df_filtered['gauge_id'].isin(train_gauge_ids)]
test_df = df_filtered[df_filtered['gauge_id'].isin(test_gauge_ids)]

# Define features and target
features = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
target = ['streamflow']

# Prepare the X and y datasets for train and test
X_train = train_df[features + ['gauge_id']]
y_train = train_df[target]

X_test = test_df[features + ['gauge_id']]
y_test = test_df[target]

# Get all unique categories in the entire dataset for 'gauge_id'
all_gauge_ids = df_filtered['gauge_id'].unique()

# Define the ColumnTransformer with predefined categories for gauge_id
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features),
        ('cat', OneHotEncoder(categories=[all_gauge_ids], drop='first'), ['gauge_id'])
    ])

# Apply the transformations to the training and test sets
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Define the number of timesteps and features
timesteps = 20  # Number of previous data points to consider
num_features = X_train_transformed.shape[1]  # Number of features after transformation

# Reshape the training and test data for BiLSTM
n_samples_train = X_train_transformed.shape[0]
n_samples_test = X_test_transformed.shape[0]

X_train_reshaped = np.zeros((n_samples_train - timesteps + 1, timesteps, num_features))
X_test_reshaped = np.zeros((n_samples_test - timesteps + 1, timesteps, num_features))

for i in range(timesteps, n_samples_train + 1):
    X_train_reshaped[i - timesteps] = X_train_transformed[i - timesteps:i]

for i in range(timesteps, n_samples_test + 1):
    X_test_reshaped[i - timesteps] = X_test_transformed[i - timesteps:i]

# Reshape the target variable accordingly
y_train_reshaped = y_train[timesteps-1:].values.ravel()
y_test_reshaped = y_test[timesteps-1:].values.ravel()

# Build the BiLSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=64, activation='relu', input_shape=(timesteps, num_features))))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='linear'))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the BiLSTM model with early stopping
history = model.fit(X_train_reshaped, y_train_reshaped,
                    epochs=40,
                    batch_size=32,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

# Make predictions on the train set
y_train_pred = model.predict(X_train_reshaped)
# Compute R² score for the train set
r2_train = r2_score(y_train_reshaped, y_train_pred)

# Make predictions on the test set
y_test_pred = model.predict(X_test_reshaped)
# Compute R² score for the test set
r2_test = r2_score(y_test_reshaped, y_test_pred)

print(f'The train R² score is: {r2_train:.4f}')
print(f'The test R² score is: {r2_test:.4f}')

# Pad zeros to match the original time series dimensions for the train data
y_train_pred_padded = np.concatenate((np.zeros(timesteps-1), y_train_pred.flatten()))
# Compute R² score for the padded train data
r2_train_padded = r2_score(y_train, y_train_pred_padded)
print(f'The R² score for the padded train data is: {r2_train_padded:.4f}')

# Pad zeros to match the original time series dimensions for the test data
y_test_pred_padded = np.concatenate((np.zeros(timesteps-1), y_test_pred.flatten()))
# Compute R² score for the padded test data
r2_test_padded = r2_score(y_test, y_test_pred_padded)
print(f'The R² score for the padded test data is: {r2_test_padded:.4f}')

<ipython-input-12-ae5eef4c9b2a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['gauge_id'] = df_filtered['gauge_id'].astype('category')
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 28s 8ms/step - loss: 1624615.5000 - val_loss: 132277.9062
Epoch 2/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 792694.3125 - val_loss: 111504.3047
Epoch 3/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 690925.3125 - val_loss: 77918.1719
Epoch 4/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 702503.5625 - val_loss: 89043.3438
Epoch 5/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 574627.6875 - val_loss: 77511.4922
Epoch 6/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 581931.7500 - val_loss: 81901.4609
Epoch 7/40
2866/2866 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 518236.5312 - val_loss: 92196.6797
Epoch 8/40
1748/2866 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 452520.7812